In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import os

In [ ]:
import tensorflow as tf
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

In [ ]:
%cd "/content/drive/My Drive"

In [ ]:
!ls

In [ ]:
%cd "/content/drive/My Drive/450"

In [ ]:
strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
print ('Number of devices: {}'.format(strategy.num_replicas_in_sync))

In [ ]:
from tensorflow.keras.layers import Layer
import matplotlib.pyplot as plt
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

In [ ]:
train_ds=tf.keras.preprocessing.image_dataset_from_directory(
    directory=r"/content/drive/My Drive/450/archive/train",
    image_size=(224,244),
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=8,
    shuffle=True,
    seed=None
)

In [ ]:
valid_ds=tf.keras.preprocessing.image_dataset_from_directory(
    directory=r"/content/drive/My Drive/450/archive/valid",
    image_size=(224,244),
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=8,
    shuffle=True,
    seed=None
)

In [ ]:
rescale = Rescaling(scale=1.0/255)
train_ds = train_ds.map(lambda image,label:(rescale(image),label))
valid_ds = valid_ds.map(lambda image,label:(rescale(image),label))

In [ ]:
BATCH_SIZE=8

In [ ]:
class VGG16(tf.keras.Model):
  def __init__(self, classes):
    super(MyModel, self).__init__()
    self._conv1a = tf.keras.layers.Conv2D(kernel_size=3, filters=16, padding='same', activation='relu')
    self._conv1b = tf.keras.layers.Conv2D(kernel_size=3, filters=30, padding='same', activation='relu')
    self._maxpool1 = tf.keras.layers.MaxPooling2D(pool_size=2)
    
    self._conv2a = tf.keras.layers.Conv2D(kernel_size=3, filters=60, padding='same', activation='relu')
    self._maxpool2 = tf.keras.layers.MaxPooling2D(pool_size=2)
    
    self._conv3a = tf.keras.layers.Conv2D(kernel_size=3, filters=90, padding='same', activation='relu')
    self._maxpool3 = tf.keras.layers.MaxPooling2D(pool_size=2)
    
    self._conv4a = tf.keras.layers.Conv2D(kernel_size=3, filters=110, padding='same', activation='relu')
    self._maxpool4 = tf.keras.layers.MaxPooling2D(pool_size=2)
    
    self._conv5a = tf.keras.layers.Conv2D(kernel_size=3, filters=130, padding='same', activation='relu')
    self._conv5b = tf.keras.layers.Conv2D(kernel_size=3, filters=40, padding='same', activation='relu')
    
    self._pooling = tf.keras.layers.GlobalAveragePooling2D()
    self._classifier = tf.keras.layers.Dense(classes, activation='softmax')

  def call(self, inputs):
    x = self._conv1a(inputs)
    x = self._conv1b(x)
    x = self._maxpool1(x)

    x = self._conv2a(x)
    x = self._maxpool2(x)

    x = self._conv3a(x)
    x = self._maxpool3(x)

    x = self._conv4a(x)
    x = self._maxpool4(x)

    x = self._conv5a(x)
    x = self._conv5b(x)

    x = self._pooling(x)
    x = self._classifier(x)
    return x

In [ ]:
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau

In [ ]:
model = VGG16(450)
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
checkpoint_path = "/content/drive/My Drive/450/archive/training_2/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
resnet.fit(train_ds,validation_data=valid_ds, epochs=5,callbacks=[cp_callback])  